In [1]:
# pip3 install ipykernel --upgrade
# python3.11.exe -m ipykernel install --user

import os
import sys
import fire
import time
import glob
import yaml
import shutil
import signal
import logging
import inspect
import functools
import statistics
import subprocess
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
from operator import xor
from pprint import pprint

import qlib
from qlib.config import C
from qlib.data import D
from qlib.workflow import R
from qlib.workflow.cli import render_template
from qlib.utils import set_log_with_config, init_instance_by_config, flatten_dict
from qlib.utils.data import update_config
from qlib.model.trainer import task_train
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.log import get_module_logger
from qlib.tests.data import GetData

set_log_with_config(C.logging_config)
logger = get_module_logger("qrun", logging.INFO)

# decorator to check the arguments
def only_allow_defined_args(function_to_decorate):
    @functools.wraps(function_to_decorate)
    def _return_wrapped(*args, **kwargs):
        """Internal wrapper function."""
        argspec = inspect.getfullargspec(function_to_decorate)
        valid_names = set(argspec.args + argspec.kwonlyargs)
        if "self" in valid_names:
            valid_names.remove("self")
        for arg_name in kwargs:
            if arg_name not in valid_names:
                raise ValueError("Unknown argument seen '%s', expected: [%s]" % (arg_name, ", ".join(valid_names)))
        return function_to_decorate(*args, **kwargs)
    return _return_wrapped

# function to handle ctrl z and ctrl c
def handler(signum, frame):
    os.system("kill -9 %d" % os.getpid())

signal.signal(signal.SIGINT, handler)

def print_class_attributes_and_methods(obj):
    print(f"Class: {obj.__class__.__name__}")
    print("Attributes and Methods:")
    for attribute in dir(obj):
        # Filter out built-in attributes and methods (those starting with '__')
        if not attribute.startswith("__"):
            try:
                # Attempt to get the value of the attribute/method
                attr_value = getattr(obj, attribute)
                if callable(attr_value):
                    print(f"{attribute} (method) -> {attr_value}")
                else:
                    print(f"{attribute} (attribute) -> {attr_value}")
            except Exception as e:
                print(f"Could not access {attribute}: {e}")

# function to calculate the mean and std of a list in the results dictionary
def cal_mean_std(results) -> dict:
    mean_std = dict()
    for fn in results:
        mean_std[fn] = dict()
        for metric in results[fn]:
            mean = statistics.mean(results[fn][metric]) if len(results[fn][metric]) > 1 else results[fn][metric][0]
            std = statistics.stdev(results[fn][metric]) if len(results[fn][metric]) > 1 else 0
            mean_std[fn][metric] = [mean, std]
    return mean_std

# function to get all the folders benchmark folder
def get_all_folders(models, exclude) -> dict:
    folders = dict()
    if isinstance(models, str):
        model_list = models.split(",")
        models = [m.lower().strip("[ ]") for m in model_list]
    elif isinstance(models, list):
        models = [m.lower() for m in models]
    elif models is None:
        models = [f.name.lower() for f in os.scandir("benchmarks")]
    else:
        raise ValueError("Input models type is not supported. Please provide str or list without space.")
    for f in os.scandir("benchmarks"):
        add = xor(bool(f.name.lower() in models), bool(exclude))
        if add:
            path = Path("benchmarks") / f.name
            folders[f.name] = str(path.resolve())
    return folders

# function to get all the files under the model folder
def get_all_files(folder_path, dataset, universe="") -> (str, str): # type: ignore
    if universe != "":
        universe = f"_{universe}"
    yaml_path = str(Path(f"{folder_path}") / f"*{dataset}{universe}.yaml")
    req_path = str(Path(f"{folder_path}") / f"*.txt")
    yaml_file = glob.glob(yaml_path)
    req_file = glob.glob(req_path)
    if len(yaml_file) == 0:
        return None, None
    else:
        return yaml_file[0], req_file[0]

# function to retrieve all the results
def get_all_results(config, dataset, folders, plot=False) -> dict:
    results = dict()
    for fn in folders:
        try:
            exp = R.get_exp(experiment_name=fn, create=False)
        except ValueError:
            # No experiment results
            continue
        recorders = exp.list_recorders()
        result = dict()
        result["annualized_return_with_cost"] = list()
        result["information_ratio_with_cost"] = list()
        result["max_drawdown_with_cost"] = list()
        result["ic"] = list()
        result["icir"] = list()
        result["rank_ic"] = list()
        result["rank_icir"] = list()
        for recorder_id in recorders:
            if recorders[recorder_id].status == "FINISHED": # type: ignore
                recorder = R.get_recorder(recorder_id=recorder_id, experiment_name=fn)
                metrics = recorder.list_metrics()
                if "1day.excess_return_with_cost.annualized_return" not in metrics:
                    print(f"{recorder_id} is skipped due to incomplete result")
                    continue
                result["annualized_return_with_cost"].append(metrics["1day.excess_return_with_cost.annualized_return"])
                result["information_ratio_with_cost"].append(metrics["1day.excess_return_with_cost.information_ratio"])
                result["max_drawdown_with_cost"].append(metrics["1day.excess_return_with_cost.max_drawdown"])
                result["ic"].append(metrics["IC"])
                result["icir"].append(metrics["ICIR"])
                result["rank_ic"].append(metrics["Rank IC"])
                result["rank_icir"].append(metrics["Rank ICIR"])
                
                if plot:
                    from qlib.contrib.report import analysis_model, analysis_position
                    print("graphical analysis: ========================================================")
                    report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
                    analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")
                    analysis_position.report_graph(report_normal_df)
                    analysis_position.risk_analysis_graph(analysis_df, report_normal_df)
                    pred_df = recorder.load_object("pred.pkl")
                    label_df = dataset.prepare("test", col_set="label")
                    label_df.columns = ["label"]
                    pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
                    analysis_position.score_ic_graph(pred_label)
                    analysis_model.model_performance_graph(pred_label)
                    # positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
                    # loaded_model = recorder.load_object("trained_model")
        results[fn] = result
    return results

# function to generate and save markdown table
def gen_and_save_md_table(metrics, dataset_name):
    table = "| Model Name | Dataset | IC | ICIR | Rank IC | Rank ICIR | Annualized Return | Information Ratio | Max Drawdown |\n"
    table += "|---|---|---|---|---|---|---|---|---|\n"
    for fn in metrics:
        ic = metrics[fn]["ic"]
        icir = metrics[fn]["icir"]
        ric = metrics[fn]["rank_ic"]
        ricir = metrics[fn]["rank_icir"]
        ar = metrics[fn]["annualized_return_with_cost"]
        ir = metrics[fn]["information_ratio_with_cost"]
        md = metrics[fn]["max_drawdown_with_cost"]
        table += f"| {fn} | {dataset_name} | {ic[0]:5.4f}±{ic[1]:2.2f} | {icir[0]:5.4f}±{icir[1]:2.2f}| {ric[0]:5.4f}±{ric[1]:2.2f} | {ricir[0]:5.4f}±{ricir[1]:2.2f} | {ar[0]:5.4f}±{ar[1]:2.2f} | {ir[0]:5.4f}±{ir[1]:2.2f}| {md[0]:5.4f}±{md[1]:2.2f} |\n"
    pprint(table)
    with open("table.md", "w") as f:
        f.write(table)
    return table

def train(config, model, dataset, id: str = "0", uri_path: str = "", experiment_name: str = "workflow"):
    """train model

    Returns
    -------
        pred_score: pandas.DataFrame
            predict scores
        performance: dict
            model performance
    """
    ana_long_short = True
    
    # start exp
    print(experiment_name,  uri_path)
    with R.start(experiment_id=id, experiment_name=experiment_name, 
                 recorder_id="0", recorder_name=f"r_{experiment_name}", 
                 uri=uri_path): # resume=True):
        # record parameters
        R.log_params(**flatten_dict(config))
        
        print("Model/Dataset Saving: ========================================================")
        R.save_objects(**{"model.pkl": model})
        R.save_objects(**{"dataset_class.pkl": dataset})
        dataset.config(dump_all=True, recursive=True) # include _data (with '_' at start)
        R.save_objects(**{"dataset_class_with_data.pkl": dataset})
        ##=============dump=============
        # dataset.to_pickle(path=f"{exp_folder_name}/dataset.pkl") # dataset is an instance of qlib.data.dataset.DatasetH
        ##=============reload=============
        # with open("dataset.pkl", "rb") as file_dataset:
        #   import pickle
        #     dataset = pickle.load(file_dataset)
        
        print("Model Fitting: ========================================================")
        model.fit(dataset)
        
        print("Model Importance: ========================================================")
        print(model.get_feature_importance())
        
        # prediction
        print("Recorder: ========================================================")
        recorder = R.get_recorder()
        pprint(R)
        pprint(recorder)
        pprint(recorder.get_local_dir()) # type: ignore
        rid = recorder.id
        print("Prediction: ========================================================")
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
        from qlib.contrib.model.xgboost import XGBModel
        # XGBModel.predict(DatasetH)
        pred_score = sr.load("pred.pkl")
        pprint(pred_score)

        # calculate ic and ric
        print("Signal Analysis: ========================================================")
        sar = SigAnaRecord(recorder,
                           ana_long_short=ana_long_short, # generate long/short data
                           ann_scaler=252, # convert daily to annual
                           )
        sar.generate()
        ic = sar.load("ic.pkl")
        ric = sar.load("ric.pkl")
        if ana_long_short:
            long_avg_r = sar.load("long_avg_r.pkl")
            long_short_r = sar.load("long_short_r.pkl")
            sig_ana = {"ic": ic, "ric": ric, "long_avg_r": long_avg_r, "long_short_r": long_short_r}
        else:
            sig_ana = {"ic": ic, "ric": ric}
        # uri_path = R.get_uri()
    return pred_score, sig_ana, rid

def backtest_analysis(pred, rid, config, uri_path: str = "", experiment_name: str = "workflow"):
    """backtest and analysis

    Parameters
    ----------
    rid : str
        the id of the recorder to be used in this function
    uri_path: str
        mlflow uri path

    Returns
    -------
    analysis : pandas.DataFrame
        the analysis result
    """
    
    with R.uri_context(uri=uri_path):
        recorder = R.get_recorder(experiment_name=experiment_name, recorder_id=rid)
        
    port_analysis_config = config.get("port_analysis_config")
    print("Backtest: ========================================================")
    # backtest
    par = PortAnaRecord(recorder, port_analysis_config, risk_analysis_freq="day")
    par.generate()
    analysis_df = par.load("port_analysis_1day.pkl")
    print(analysis_df)
    return analysis_df

def collect_results(config, dataset, exp_folder_name, dataset_name, plot):
    folders = get_all_folders(exp_folder_name, dataset_name)
    # getting all results
    sys.stderr.write(f"Retrieving results...\n")
    results = get_all_results(config, dataset, folders, plot)
    if len(results) > 0:
        # calculating the mean and std
        sys.stderr.write(f"Calculating the mean and std of results...\n")
        results = cal_mean_std(results)
        # generating md table
        sys.stderr.write(f"Generating markdown table...\n")
        gen_and_save_md_table(results, dataset_name)
        sys.stderr.write("\n")
    sys.stderr.write("\n")

class ModelRunner:
    # def __init__(self):
    #     self.run()
        
    def _init_qlib(self, exp_folder_name, uri_path):
        # init qlib
        provider_uri = "./.qlib/qlib_data/cn_data"
        # config["qlib_init"]["provider_uri"]
        # config["qlib_init"]["region"]
        GetData().qlib_data(
          name="qlib_data",
          target_dir=provider_uri,
          interval="1d",
          region="cn",
          exists_skip=True
          )
        
        # p = Path("./.qlib/qlib_data/cn_data/financial").expanduser()
        # if not p.exists():
        #     !cd ../../scripts/data_collector/pit/ && pip install -r requirements.txt
        #     !cd ../../scripts/data_collector/pit/ && python collector.py download_data --source_dir ./.qlib/stock_data/source/pit --start 2000-01-01 --end 2020-01-01 --interval quarterly --symbol_regex "^(600519|000725).*"
        #     !cd ../../scripts/data_collector/pit/ && python collector.py normalize_data --interval quarterly --source_dir ./.qlib/stock_data/source/pit --normalize_dir ./.qlib/stock_data/source/pit_normalized
        #     !cd ../../scripts/ && python dump_pit.py dump --csv_path ./.qlib/stock_data/source/pit_normalized --qlib_dir ./.qlib/qlib_data/cn_data --interval quarterly
        
        qlib.init(
            exp_manager={
                "class": "MLflowExpManager",
                "module_path": "qlib.workflow.expm",
                "kwargs": {
                    "uri": uri_path,
                    "default_exp_name": "Experiment",
                },
            }
        )

    # function to run the all the models
    @only_allow_defined_args
    def run(
        self,
        models_name=['xgboost'], # None,
        dataset_name="Alpha158",
        universe="",
        exclude=False,
        exp_folder_name: str = "mlruns",
        plot: bool = False
    ):
        """
        models="lightgbm", dataset="Alpha158", universe="csi500" will result in running the following config:
        benchmarks/LightGBM/workflow_config_lightgbm_Alpha158_csi500.yaml

        Parameters:
        -----------
        models : str or list
            determines the specific model or list of models to run or exclude.
        exclude : boolean
            determines whether the model being used is excluded or included.
        dataset : str
            determines the dataset to be used for each model.
        universe : str
            the stock universe of the dataset.
            default "" indicates that
        exp_folder_name: str
            the name of the experiment folder

            # Case 7 - run lightgbm model on csi500.
            python run_all_model.py run 3 lightgbm Alpha158 csi500

        """
        uri_path = "file:" + str(Path(os.getcwd()).joinpath(exp_folder_name).resolve())
        self._init_qlib(exp_folder_name, uri_path)

        # get all folders
        folders = get_all_folders(models_name, exclude)
        # run all the model for iterations
        for idx, fn in enumerate(folders):
            print(fn, folders)
            # get all files
            sys.stderr.write("Retrieving files...\n")
            yaml_path, req_path = get_all_files(folders[fn], dataset_name, universe=universe)
            if yaml_path is None:
                sys.stderr.write(f"There is no {dataset_name}.yaml file in {folders[fn]}")
                continue
            sys.stderr.write("\n")
            
            # Render the template
            rendered_yaml = render_template(yaml_path)
            config = yaml.safe_load(rendered_yaml)
            
            # model initialization
            print("Model: ========================================================")
            model = init_instance_by_config(config["task"]["model"])
            pprint(model)
            print("Dataset: ========================================================")
            dataset = init_instance_by_config(config["task"]["dataset"])
            pprint(dataset)
            
            print("Dataset_Handler: ========================================================")
            hd = dataset.handler
            print("features/labels: ", hd.fetch(col_set="__all", data_key="infer")) # raw/learn/infer
            # print("feature/label formula: ", hd.data_loader.fields)
            print("train_processor: ", hd.learn_processors)
            print("infer_processor: ", hd.infer_processors)
            print("process_type: ", hd.process_type) # independent/append (dictates how processors apply on datesets)
            
            pred, sig_ana, rid = train(config, model, dataset, str(idx+1), uri_path, experiment_name=fn)
            # assert ic/ric >= 0
            
            analyze_df = backtest_analysis(pred, rid, config, uri_path, experiment_name=fn)
            # assert "excess_return_with_cost"/"annualized_return" >= 0.05
            # assert not analyze_df.isna().any().any()
        
        plot = False
        analysis = True
        rename = False
        print("Collect Result: ========================================================")
        if analysis:
          collect_results(config, dataset, exp_folder_name, dataset_name, plot)
          shutil.move("table.md", f"{exp_folder_name}/table.md")
        
        if rename:
          # move results folder
          folder_with_stamp = exp_folder_name + f"_{dataset_name}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
          shutil.move(exp_folder_name, folder_with_stamp)
          
if __name__ == "__main__":
    # fire.Fire(ModelRunner)  # run all the model
    runner = ModelRunner()
    runner.run()


2024-08-20 15:12:05.706 | WARNING  | qlib.tests.data:qlib_data:195 - Data already exists: ./.qlib/qlib_data/cn_data, the data download will be skipped
	If downloading is required: `exists_skip=False` or `change target_dir`
[28124:MainThread](2024-08-20 15:12:05,708) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[28124:MainThread](2024-08-20 15:12:05,713) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[28124:MainThread](2024-08-20 15:12:05,714) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/chuyin.wang/Desktop/share/fin/trader/run/train/.qlib/qlib_data/cn_data')}


XGBoost {'XGBoost': 'C:\\Users\\chuyin.wang\\Desktop\\share\\fin\\trader\\run\\train\\benchmarks\\XGBoost'}


Retrieving files...

[28124:MainThread](2024-08-20 15:12:05,724) INFO - qlib.qrun - [cli.py:78] - Render the template with the context: {}


Model: ========================================================
Dataset: ========================================================


[28124:MainThread](2024-08-20 15:13:20,572) INFO - qlib.timer - [log.py:127] - Time cost: 71.535s | Loading data Done
[28124:MainThread](2024-08-20 15:13:20,830) INFO - qlib.timer - [log.py:127] - Time cost: 0.087s | DropnaLabel Done
[28124:MainThread](2024-08-20 15:13:21,463) INFO - qlib.timer - [log.py:127] - Time cost: 0.632s | CSZScoreNorm Done
[28124:MainThread](2024-08-20 15:13:21,476) INFO - qlib.timer - [log.py:127] - Time cost: 0.901s | fit & process data Done
[28124:MainThread](2024-08-20 15:13:21,477) INFO - qlib.timer - [log.py:127] - Time cost: 72.440s | Init data Done


...

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.001114
std                0.016669
annualized_return  0.265098
information_ratio  1.030870
max_drawdown      -0.171983
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean               0.000203
std                0.006099
annualized_return  0.048316
information_ratio  0.513509
max_drawdown      -0.039758
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean               0.000006
std                0.006095
annualized_return  0.001323
information_ratio  0.014066
max_drawdown      -0.045454


[28124:MainThread](2024-08-20 15:14:05,337) INFO - qlib.workflow - [record_temp.py:540] - Indicator analysis record 'indicator_analysis_1day.pkl' has been saved as the artifact of the Experiment 315313931788076689


'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0
                                                  risk
excess_return_without_cost mean               0.000203
                           std                0.006099
                           annualized_return  0.048316
                           information_ratio  0.513509
                           max_drawdown      -0.039758
excess_return_with_cost    mean               0.000006
                           std                0.006095
                           annualized_return  0.001323
                           information_ratio  0.014066
                           max_drawdown      -0.045454
Collect Result: ========================================================


Retrieving results...
Calculating the mean and std of results...
Generating markdown table...


('| Model Name | Dataset | IC | ICIR | Rank IC | Rank ICIR | Annualized Return '
 '| Information Ratio | Max Drawdown |\n'
 '|---|---|---|---|---|---|---|---|---|\n'
 '| XGBoost | Alpha158 | 0.0178±0.00 | 0.1353±0.00| 0.0252±0.00 | 0.2118±0.00 '
 '| 0.0013±0.00 | 0.0141±0.00| -0.0455±0.00 |\n')
